## Use OpenAI OSS models on Amazon Bedrock with LangChain

In this notebook, you learn how you can use OpenAI OSS models from Bedrock with Langchain. 

## Setup

First we need to install LangChain.

In [1]:
%pip install -q langchain langchain_aws langchain_community --upgrade


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Model Invocation

OpenAI gpt-oss-120b and gpt-oss-20b models are designed for text generation and reasoning tasks. Their Bedrock model ids are:

gpt-oss-120b model: `openai.gpt-oss-120b-1:0`  
gpt-oss-20b model: `openai.gpt-oss-20b-1:0`


In [2]:
# You can change model ID here

MODEL_ID = "openai.gpt-oss-20b-1:0"

### Text Generation

In [3]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatBedrockConverse(
    model_id=MODEL_ID,
    temperature=0.7
)

messages = [
    SystemMessage(content="You are an author with experience writing creative novels"),
    HumanMessage(
        content="Write an outlin for a novel about a wizard named Theodore graduating from college"
    ),
]

response = llm.invoke(messages)
print(f"Request ID: {response.id}")
response.pretty_print()


Request ID: run--d802b7e6-6b8d-49ed-b376-1b4b042047a0-0
================================== Ai Message ==================================

[{'type': 'reasoning_content', 'reasoning_content': {'text': 'We need to produce an outline for a novel about a wizard named Theodore graduating from college. The outline should probably include main plot points, chapters, arcs, character development, themes, structure. Should be creative and imaginative. The wizard is graduating from college—maybe a wizarding college? Or a regular college where he is studying? How to integrate wizardry? Maybe he discovers his powers? Maybe it\'s at Hogwarts? But it\'s "graduating from college" suggests it\'s a normal college. The wizard may have to balance being a wizard with normal life. Or it\'s a magical college. Could be a contemporary YA, or adult fantasy. We need to outline: major beats, chapters, subplots, character arcs, conflict, resolution. Provide a detailed outline with section headings, maybe chapters, 

In [4]:
# Here we can pass the chat history to the model to ask follow up questions
multi_turn_messages = [
    *messages,
    response,
    HumanMessage(content="Select your favorite and tell me why"),
]

response = llm.invoke(multi_turn_messages)
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()



Request ID: run--3c0d5c91-eea8-40e7-b073-877d30f651de-0
================================== Ai Message ==================================

[{'type': 'reasoning_content', 'reasoning_content': {'text': 'The user says: "Select your favorite and tell me why". The conversation: The user asked earlier for an outline for a novel about a wizard named Theodore graduating from college. The assistant provided an outline. Now the user says "Select your favorite and tell me why". The user presumably wants the assistant to choose from among the outlines or scenarios? The assistant only provided one outline. So maybe the user wants the assistant to select their favorite from possible outlines? But only one outline was given. So perhaps the user refers to an earlier set of options? Actually the assistant gave a single outline. So maybe the user wants the assistant to pick their favorite among the outline\'s subplots or something? The user might be asking: "Select your favorite [part of the outline] a

### Structured Output

Structured output is a great way to force the model to return in a specific way. We use Greedy Decoding params here for more determinist results (Temperature = 1, Top P = 1, Top K = 1)

## Agentic Workflows

The OpenAI OSS models are capable of handling tool calling and agentic workflows.

#### Binding Tools

When using a model for tool calling you can take advantage of the bind tools method. This will pass a formatted tool config to the model. We recommend when taking advantage of tool calling or agentic workflows to use greedy decoding values. This means temperature=1, topP=1, topK=1

In [6]:
from langchain_aws import ChatBedrockConverse
from langchain.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers. Use tools for arithmetic operations"""
    return a * b

tools = [multiply]

llm_with_tools = ChatBedrockConverse(
    model=MODEL_ID,
    temperature=1,
    top_p=1
).bind_tools(tools)

response = llm_with_tools.invoke([("user", "What is 8*8")])

print("[Model Response]\n")
print(response.content)

print("\n[Tool Calls]\n")
print(response.tool_calls)

[Model Response]

[{'type': 'reasoning_content', 'reasoning_content': {'text': 'We need to call the function multiply.', 'signature': ''}}, {'type': 'text', 'text': ''}, {'type': 'tool_use', 'name': 'multiply', 'input': {'a': 8, 'b': 8}, 'id': 'tooluse_36Ca4nLHRb6XgPfj-zwWsw'}]

[Tool Calls]

[{'name': 'multiply', 'args': {'a': 8, 'b': 8}, 'id': 'tooluse_36Ca4nLHRb6XgPfj-zwWsw', 'type': 'tool_call'}]


#### Tool Calling Agents

For full workflows you can take advantage of custom parsers that will intercept outputs of the stream and allow you to invoke tools

In [7]:
from langchain.agents import tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrockConverse

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools = [multiply]

llm_with_tools = ChatBedrockConverse(
    model=MODEL_ID,
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
).bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Use tools for performing arithmetic operations.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is 2*2?"})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 2, 'b': 2}`
responded: [{'type': 'reasoning_content', 'reasoning_content': {'text': 'The user asks: What is 2*2? We should use the multiply tool.', 'signature': ''}}, {'type': 'text', 'text': ''}, {'type': 'tool_use', 'name': 'multiply', 'input': {'a': 2, 'b': 2}, 'id': 'tooluse_hh95siWTRHWFUpK6RkIIaw'}]

4[{'type': 'reasoning_content', 'reasoning_content': {'text': 'We got the result.', 'signature': ''}}, {'type': 'text', 'text': '2\u202f×\u202f2\u202f=\u202f4'}]

> Finished chain.


{'input': 'What is 2*2?',
 'output': [{'type': 'reasoning_content',
   'reasoning_content': {'text': 'We got the result.', 'signature': ''}},
  {'type': 'text', 'text': '2\u202f×\u202f2\u202f=\u202f4'}]}

### Use with MCP Server

We have a simple MCP Server for demonstration.

In [8]:
!pygmentize math_mcp_server.py 

"""
Math MCP Server

This module implements a Model Context Protocol (MCP) server that provides mathematical
capabilities to AI assistants. The server exposes tools for basic arithmetic operations,
prompts for mathematical assistance, and resources for configuration and greetings.

Features:
- Mathematical tools: addition, subtraction, multiplication, and division operations
- Prompts: math assistant and system prompts for AI interactions
- Resources: personalized greetings and application configuration

The server runs via stdio and can be integrated with MCP-compatible AI systems
to extend their mathematical reasoning capabilities.
"""

from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Math")

# Prompts
@mcp.prompt(title='Example Prompt')
def example_prompt(question: str) -> str:
    """ math assistant prompt with detailed problem-solving guidance"""
    return f"""
    You are an expert mathematics assistant with access to computational tools.
    
    Your approach to solving 

In [9]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio

# Math Server Parameters
server_params = StdioServerParameters(
    command="python3",
    args=["math_mcp_server.py"],
    env=None,
)

async def main():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            
            # Grab the list of available prompts
            response = await session.list_prompts()
            print("\n---------------- PROMPTS ----------------")
            counter = 1
            for prompt in response.prompts:
                print(f'\nPrompt {counter}')
                print(f'Name: {prompt.name}')
                print(f'Title: {prompt.title}')
                print(f'Description: {prompt.description}')
                counter += 1


            # Grab the list of available tools
            response = await session.list_tools()
            print("\n---------------- TOOLS ----------------")
            counter = 1
            for tool in response.tools:
                print(f'\nTool {counter}')
                print(f'Name: {tool.name}')
                print(f'Title: {tool.title}')
                print(f'Description: {tool.description}')
                
                counter += 1

            # Use example prompt to prepare prompt with example 
            prompt = await session.get_prompt("example_prompt", arguments={"question": "what is 8+72"})
            print("\n---------------- EXAMPLE PROMPT ----------------")
            print(prompt.messages[0].content.text)
            


            # Invoke add method on MCP Server

            
            result = await session.call_tool("multiply", arguments={"a": 60, "b": 20})
            print("\n---------------- RESPONSE ----------------")
            print('Invoking multiply tool with arguments a=60 and b=20')
            
            
            print(f'Result: {result.content[0].text}')

In [10]:
await main()


---------------- PROMPTS ----------------

Prompt 1
Name: example_prompt
Title: Example Prompt
Description: math assistant prompt with detailed problem-solving guidance

Prompt 2
Name: system_prompt
Title: System Prompt
Description: System prompt for mathematical AI assistant with tool usage guidance

---------------- TOOLS ----------------

Tool 1
Name: add
Title: Add tool
Description: 
Performs addition of two integers with validation and error handling.

This tool computes the sum of two integer values and returns the result.
It's designed for accurate arithmetic operations in mathematical problem-solving.

Args:
    a (int): The first integer operand (addend)
    b (int): The second integer operand (addend)

Returns:
    int: The sum of a and b (a + b)

Examples:
    add(5, 3) -> 8
    add(-2, 7) -> 5
    add(0, 100) -> 100

Use this tool when you need to:
- Calculate sums in mathematical expressions
- Verify addition operations step-by-step
- Ensure accuracy in arithmetic computa

## Conclusion

In this notebook, we explored the capabilities of OpenAI OSS models. These capabilities include text generation, tool calling, using MCP servers etc.